# Project 6: NMF

Select a basic, explicit feedback dataset, many examples here:

<https://gist.github.com/entaroadun/1653794>

<https://github.com/caserec/Datasets-for-Recommender-Systems>

<https://github.com/RUCAIBox/RecSysDatasets>

And use the Surprise library:

<http://surpriselib.com/>

To implement a basic recommendation system. Many of those datasets are already loaded into the Surprise package to make this easy. You should tune and cross validate your system to select the best values for the # of latent dimensions, the regularization parameter, and any other hyperparameters.

Stretch Goal #1 (3 points)

Using an explicit feedback dataset, implement the NMF algorithm by hand, tune it via cross validation to select the # of latent dims and regularization parameter

Stretch goal #2 (3 points)

Using an implicit feedback dataset, some can be found here:

<https://cseweb.ucsd.edu/~jmcauley/datasets.html>

And implement the implicit feedback version of NMF. Evaluating performance will be harder however, so instead of doing a grid search and tuning, try to output the ratings explanations (i.e. use eqn 7 of the implicit feedback dataset).  Nobody has ever attempted this stretch goal.


In [1]:
from surprise import NMF, Dataset, accuracy
from surprise.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

In [2]:
# Load the MovieLens 100k dataset
data = Dataset.load_builtin('ml-100k', prompt=False)
data.raw_ratings[:5] # columns: user_id, item_id, rating, timestamp

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596')]

In [3]:
# Tune hyperparameters with GridSearchCV class from Surprise
param_grid = {
    'n_factors': list(range(10, 60, 10)), # Number of latent factors
    'n_epochs': list(range(10, 40, 10)), # Number of epochs
    'reg_pu': np.linspace(0.01, 0.2, 5).tolist(), # Regularization for user factors
    'reg_qi': np.linspace(0.01, 0.2, 5).tolist(), # Regularization for item factors
    'biased': [True, False] # Include user/item bias terms
}

gs = GridSearchCV(NMF, param_grid, measures=['rmse'], cv=5, n_jobs=-1) # cv=5 is 5-fold cross-validation
gs.fit(data)
best_params = gs.best_params['rmse']
print("Best parameters:", best_params)
print("Best RMSE:", gs.best_score['rmse'])

Best parameters: {'n_factors': 10, 'n_epochs': 30, 'reg_pu': 0.2, 'reg_qi': 0.01, 'biased': True}
Best RMSE: 0.9369756933233495


In [4]:
# Train the system
test_size = 0.2
trainset, testset = train_test_split(data, test_size=test_size)
algo = NMF(**best_params)
algo.fit(trainset)
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions, verbose=False)
print("Test RMSE:", rmse)

Test RMSE: 0.9370796130578684


In [5]:
# Recommend items for a specific user
user_id = '196'  # MovieLens user ID

all_items = set(iid for (_, iid, _, _) in data.raw_ratings)
rated_items = set(iid for (uid, iid, _, _) in data.raw_ratings if uid == str(user_id))
unrated_items = list(all_items - rated_items)
predictions = [(item, algo.predict(str(user_id), item).est) for item in unrated_items]
recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:5]

print(f"Top recommendations for user {user_id}:", [item for item, _ in recommendations])

Top recommendations for user 196: ['1554', '1467', '1358', '1631', '1449']


In [6]:
def naive_rmse(data, user_id):
    """
    Calculate RMSE for the naive recommendation system.
    """
    # Compute global average for all items
    global_average = np.mean([float(rating) for (_, _, rating, _) in data.raw_ratings])

    # Get test ratings for the user
    test_ratings = [(uid, iid, float(rating)) for (uid, iid, rating, _) in data.raw_ratings if uid == str(user_id)]

    # Calculate RMSE for naive predictions
    errors = [(rating - global_average) ** 2 for (_, _, rating) in test_ratings]
    rmse = np.sqrt(np.mean(errors))
    return rmse

In [7]:
naive_rmse_score = naive_rmse(data, user_id)

print("Test RMSE (Surprise):", rmse)
print("Test RMSE (Naive):", naive_rmse_score)

Test RMSE (Surprise): 0.9370796130578684
Test RMSE (Naive): 1.0065940689274897


## Stretch Goal 1: NMF by hand

In [8]:
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import itertools

In [9]:
# Load the MovieLens 100k dataset
url = "http://files.grouplens.org/datasets/movielens/ml-100k/u.data"
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
data = pd.read_csv(url, sep='\t', names=column_names)
data


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [10]:
data = data.drop(columns=['timestamp'])

# Adjust user_id and item_id to start from 0 (zero-indexed)
data['user_id'] -= 1
data['item_id'] -= 1

# Split the data into training and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Create a sparse matrix for faster computations (reference: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)
n_users = data['user_id'].max() + 1
n_items = data['item_id'].max() + 1
train_matrix = csr_matrix(
    (train_data['rating'], (train_data['user_id'], train_data['item_id'])),
    shape=(n_users, n_items)
)
train_matrix.toarray()

array([[0, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [11]:
def als_reconstruct(sparse_matrix, n_factors=10, iterations=10, reg=0.1):
    """
    Alternating Least Squares for matrix reconstruction.
    
    Parameters:
    - sparse_matrix: The user-item matrix (CSR format).
    - n_factors: Number of latent factors.
    - iterations: Number of ALS iterations.
    - reg: Regularization term.
    
    Returns:
    - reconstructed_matrix: The predicted user-item matrix.
    """
    n_users, n_items = sparse_matrix.shape
    user_factors = np.random.rand(n_users, n_factors)
    item_factors = np.random.rand(n_items, n_factors)
    
    for _ in range(iterations):
        # Fix item factors and update user factors
        for u in range(n_users):
            non_zero = sparse_matrix[u].indices
            if len(non_zero) > 0:
                A = item_factors[non_zero].T @ item_factors[non_zero] + reg * np.eye(n_factors)
                b = sparse_matrix[u, non_zero].toarray().flatten() @ item_factors[non_zero]
                user_factors[u] = np.linalg.solve(A, b)
        
        # Fix user factors and update item factors
        for i in range(n_items):
            non_zero = sparse_matrix[:, i].indices
            if len(non_zero) > 0:
                A = user_factors[non_zero].T @ user_factors[non_zero] + reg * np.eye(n_factors)
                b = sparse_matrix[non_zero, i].toarray().flatten() @ user_factors[non_zero]
                item_factors[i] = np.linalg.solve(A, b)
    
    return user_factors @ item_factors.T

predicted_matrix = als_reconstruct(train_matrix, n_factors=10, iterations=10, reg=0.1)
print("train matrix")
print(train_matrix.toarray())
print("predicted matrix")
print(predicted_matrix)

train matrix
[[0 3 4 ... 0 0 0]
 [4 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [5 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 5 0 ... 0 0 0]]
predicted matrix
[[0.         2.99949834 3.99900161 ... 0.         0.         2.90625753]
 [0.         2.90628561 3.87472821 ... 0.         0.         2.81594236]
 [0.         2.40898983 3.2117218  ... 0.         0.         2.33410525]
 ...
 [0.         2.75123703 3.66801381 ... 0.         0.         2.66571354]
 [0.         3.11214712 4.14918761 ... 0.         0.         3.01540457]
 [0.         2.8554442  3.80694525 ... 0.         0.         2.76668138]]


In [12]:
def calculate_rmse(test_data, predicted_matrix):
    errors = []
    for _, row in test_data.iterrows():
        user, item, actual = int(row['user_id']), int(row['item_id']), row['rating']
        predicted = predicted_matrix[user, item]
        errors.append((actual - predicted) ** 2)
    return np.sqrt(np.mean(errors))

rmse = calculate_rmse(test_data, predicted_matrix)
print("RMSE:", rmse)


RMSE: 3.0639695078899045


In [ ]:
def cross_validate_als(data, param_comb, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    rmse_list = []
    
    for train_index, val_index in kf.split(data):
        train_data = data.iloc[train_index]
        val_data = data.iloc[val_index]
        
        train_matrix = csr_matrix(
            (train_data['rating'], (train_data['user_id'], train_data['item_id'])),
            shape=(n_users, n_items)
        )
        
        predicted_matrix = als_reconstruct(
            train_matrix,
            n_factors=param_comb['n_factors'],
            iterations=param_comb['iterations'],
            reg=param_comb['reg']
        )
        
        rmse = calculate_rmse(val_data, predicted_matrix)
        rmse_list.append(rmse)
    
    return np.mean(rmse_list)

# Define grid search parameters
param_grid = {
    'n_factors': [10, 50],
    'reg': [0.01, 0.1],
    'iterations': [5, 50]
}

best_params = None
best_rmse = float('inf')
# Iterate over all parameter combinations
for comb in itertools.product(*param_grid.values()):
    param_comb = dict(zip(param_grid.keys(), comb))
    print(f"Testing parameters: {param_comb}")
    
    avg_rmse = cross_validate_als(data, param_comb)
    print(f"Average RMSE: {avg_rmse}")
    
    if avg_rmse < best_rmse:
        best_rmse = avg_rmse
        best_params = param_comb

print(f"Best Parameters: {best_params}")
print(f"Best RMSE: {best_rmse}")


Testing parameters: {'n_factors': 10, 'reg': 0.01, 'iterations': 5}
Average RMSE: 3.05089057074541
Testing parameters: {'n_factors': 10, 'reg': 0.01, 'iterations': 50}
Average RMSE: 3.05092776527504
Testing parameters: {'n_factors': 10, 'reg': 0.1, 'iterations': 5}
Average RMSE: 3.0492360015130227
Testing parameters: {'n_factors': 10, 'reg': 0.1, 'iterations': 50}
Average RMSE: 3.0491201992159365
Testing parameters: {'n_factors': 50, 'reg': 0.01, 'iterations': 5}
Average RMSE: 3.0508511312388853
Testing parameters: {'n_factors': 50, 'reg': 0.01, 'iterations': 50}
Average RMSE: 3.050918820576931
Testing parameters: {'n_factors': 50, 'reg': 0.1, 'iterations': 5}
Average RMSE: 3.048963074454243
Testing parameters: {'n_factors': 50, 'reg': 0.1, 'iterations': 50}
Average RMSE: 3.0489864929846604
Best Parameters: {'n_factors': 50, 'reg': 0.1, 'iterations': 5}
Best RMSE: 3.048963074454243
